In [ ]:
import pygame
import sys
import copy

# Game configuration
gameRows = 15
gameCols = 15
cellSize = 45
width = gameRows * cellSize
height = gameCols * cellSize
windowWidth = width + 200
cellCenter = cellSize // 2

# Colors
blackLine = (0, 0, 0)
backgroundColor = (245, 222, 179)
player_color = (0, 0, 0)
AI_player = (255, 255, 255)

# Menu and difficulty settings
isMenu = True
easy_mode = False
depth = 0

pygame.font.init()
font = pygame.font.SysFont("Arial", 24)

def startMenu(window):
    # Set the x and y coordinates for the Easy button
    easyX, easyY = windowWidth // 2 - 60, height // 2

    # Set the y coordinate for the Medium button (60 pixels below Easy)
    mediumY = height // 2 + 60

    # Set the y coordinate for the Hard button (60 pixels below Medium)
    hardY = height // 2 + 120

    # Fill the entire window with the background color
    window.fill(backgroundColor)

    # Render the game title text "Animus Gomoko Game" in black
    difficultyText = font.render("Animus Gomoko Game", True, (0, 0, 0))

    # Display the game title text on the screen
    window.blit(difficultyText, (windowWidth // 3 + 40, height // 4 - 50))

    # Render the subtitle text "Choose your difficulty" in gray
    difficultyText = font.render("Choose your difficulty", True, (100, 100, 100))

    # Display the subtitle text on the screen
    window.blit(difficultyText, (windowWidth // 3 + 40, height // 3 + 5))

    # ------------------ EASY BUTTON ------------------

    # Draw a white rectangle to represent the Easy button
    pygame.draw.rect(window, (255, 255, 255), (windowWidth // 2 - 60, height // 2, 120, 50), 0)

    # Render the text "Easy" in blue
    easyText = font.render("Easy", True, (0, 10, 250))

    # Display the "Easy" text on top of the Easy button
    window.blit(easyText, (windowWidth // 2 - 30, height // 2 + 5))

    # ------------------ MEDIUM BUTTON ------------------

    # Draw a white rectangle to represent the Medium button
    pygame.draw.rect(window, (255, 255, 255), (windowWidth // 2 - 60, height // 2 + 60, 120, 50), 0)

    # Render the text "Medieum" in blue (Note: typo in the text "Medieum" should be "Medium")
    easyText = font.render("Medieum", True, (0, 10, 250))

    # Display the "Medieum" text on top of the Medium button
    window.blit(easyText, (windowWidth // 2 - 45, height // 2 + 65))

    # ------------------ HARD BUTTON ------------------

    # Draw a white rectangle to represent the Hard button
    pygame.draw.rect(window, (255, 255, 255), (windowWidth // 2 - 60, height // 2 + 120, 120, 50), 0)

    # Render the text "Hard" in blue
    easyText = font.render("Hard", True, (0, 10, 250))

    # Display the "Hard" text on top of the Hard button
    window.blit(easyText, (windowWidth // 2 - 30, height // 2 + 125))

    # ------------------ LOGO IMAGE ------------------

    # Load the image file named "gameLogo.png" into a surface
    logo = pygame.image.load("gameLogo.png")

    # Display the logo at the top-left corner of the screen (0, 0)
    window.blit(logo, (0, 0))

    # Return the coordinates of the Easy, Medium, and Hard buttons for click detection
    return easyX, easyY, mediumY, hardY

def drawResult(board):
    isDraw=True
    for row in range(gameRows):
        for col in range(gameCols):
            if board[row][col]!=0:
                isDraw=True
            else: 
                isDraw=False
                break
    return isDraw

def evaluate_board(board, player, easy_mode):
    # If easy_mode is enabled, use a simple scoring method
    if easy_mode:
        score = 0  # Initialize score to 0
        opponent = 1 if player == 2 else 2  # Determine the opponent player

        # Loop through all rows and columns on the board
        for row in range(gameRows):
            for col in range(gameCols):
                # Add 1 to score for each player's piece
                if board[row][col] == player:
                    score += 1
                # Subtract 1 from score for each opponent's piece
                elif board[row][col] == opponent:
                    score -= 1
        return score  # Return the final score for easy mode

    else:
        # More advanced evaluation for non-easy mode (AI mode)
        score = 0  # Initialize score
        opponent = 1 if player == 2 else 2  # Determine opponent

        # Define directions to check: vertical, horizontal, and two diagonals (both directions)
        directions = [(1, 0), (0, 1), (1, 1), (1, -1), (-1, 1), (-1, -1)]

        # Loop through every cell on the board
        for row in range(gameRows):
            for col in range(gameCols):
                # Skip empty cells
                if board[row][col] == 0:
                    continue

                # Identify which player owns the current cell
                current_player = board[row][col]

                # Check all defined directions from the current cell
                for rowDirect, columnDirect in directions:
                    consecutive = 1  # Start with one stone
                    open_ends = 0  # Track how many ends are open (i.e., unblocked)

                    # ------------------ FORWARD DIRECTION ------------------
                    r, c = row + rowDirect, col + columnDirect
                    while 0 <= r < gameRows and 0 <= c < gameCols and board[r][c] == current_player:
                        consecutive += 1  # Count consecutive pieces
                        r += rowDirect  # Move further in the same direction
                        c += columnDirect
                    # Check if the next cell in direction is empty (open end)
                    if 0 <= r < gameRows and 0 <= c < gameCols and board[r][c] == 0:
                        open_ends += 1

                    # ------------------ BACKWARD DIRECTION ------------------
                    r, c = row - rowDirect, col - columnDirect
                    while 0 <= r < gameRows and 0 <= c < gameCols and board[r][c] == current_player:
                        consecutive += 1  # Count consecutive pieces in opposite direction
                        r -= rowDirect  # Move further back
                        c -= columnDirect
                    # Check if the previous cell in opposite direction is empty (open end)
                    if 0 <= r < gameRows and 0 <= c < gameCols and board[r][c] == 0:
                        open_ends += 1

                    # ------------------ SCORING ------------------
                    # If the line is owned by the player
                    if current_player == player:
                        if consecutive >= 5:
                            score += 100000  # Winning line, high score
                        else:
                            # Score increases with longer sequences and more open ends
                            score += (10 ** consecutive) * (open_ends + 1)
                    else:
                        # If opponent has a strong line, subtract points
                        if consecutive >= 5:
                            score -= 100000  # Opponent is winning
                        else:
                            # Decrease score based on opponent's potential
                            score -= (10 ** consecutive) * (open_ends + 1)

        return score  # Return the final calculated score

def nextmoves(board):
    moves = set()  # Use a set to store unique potential moves (no duplicates)

    # Loop through every cell in the game board
    for row in range(gameRows):
        for col in range(gameCols):
            # If the current cell is not empty (i.e., a piece is placed here)
            if board[row][col] != 0:
                # Check all 8 surrounding directions including diagonals
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        # Skip the center cell (dx == 0 and dy == 0), as it's already occupied
                        r, c = row + dx, col + dy  # Calculate the coordinates of the neighboring cell

                        # Check if the new position is within board boundaries and is empty
                        if 0 <= r < gameRows and 0 <= c < gameCols and board[r][c] == 0:
                            moves.add((r, c))  # Add the valid empty neighboring cell as a potential move

    return list(moves)  # Convert the set to a list and return

def oneWinMove(board, turn):
    # Get a list of all possible next moves (adjacent to current pieces)
    movesList = nextmoves(board)

    # Loop through each possible move
    for move in movesList:
        row, col = move  # Unpack the row and column of the current move

        # Create a deep copy of the board to simulate the move
        new_board = [r.copy() for r in board]

        # Place the current player's stone at the simulated position
        new_board[row][col] = turn

        # Check if this move leads to a win for the current player
        if checkWinner(new_board) == turn:
            return (row, col)  # If it does, return the winning move coordinates

    return None  # If no winning move is found, return None
    
def minimax(board, max_turn, depth, alpha, beta, easy_mode):
    # Check if someone has already won the game
    winner = checkWinner(board)
    if winner != 0:
        # If AI (player 2) wins, return a high positive score; if human (player 1), return a high negative score
        return None, 100000 if winner == 2 else -100000

    # If the search has reached the maximum depth, evaluate the current board state heuristically
    if depth == 0:
        return None, evaluate_board(board, 2, easy_mode)

    # Generate all possible next valid moves
    possible_moves = nextmoves(board)

    # If there are no valid moves (board full or no nearby cells), return score 0 (draw/stalemate)
    if not possible_moves:
        return None, 0

    # Max turn: AI is trying to maximize the score
    if max_turn == 2:
        best_score = -100000  # Start with a very low score
        best_move = possible_moves[0]  # Default to the first move in the list
        for move in possible_moves:
            # Simulate the move by copying the board and placing the AI's piece
            new_board = [r.copy() for r in board]
            new_board[move[0]][move[1]] = 2

            # Recursively call minimax assuming the next move is by the human (player 1)
            _, score = minimax(new_board, 1, depth - 1, alpha, beta, easy_mode)

            # If this move has a better score, update the best score and best move
            if score > best_score:
                best_score = score
                best_move = move

            # Update alpha (best already explored option for the maximizer)
            alpha = max(alpha, best_score)

            # Alpha-Beta pruning: cut off search if beta ≤ alpha (only if not in easy mode)
            if not easy_mode and beta <= alpha:
                break
        return best_move, best_score

    else:  # Min turn: human is trying to minimize the score
        best_score = 100000  # Start with a very high score
        best_move = possible_moves[0]  # Default to the first move
        for move in possible_moves:
            # Simulate the human's move
            new_board = [r.copy() for r in board]
            new_board[move[0]][move[1]] = 1

            # Recursively call minimax for the AI's next move
            _, score = minimax(new_board, 2, depth - 1, alpha, beta, easy_mode)

            # If this move leads to a lower score, update the best score and move
            if score < best_score:
                best_score = score
                best_move = move

            # Update beta (best already explored option for the minimizer)
            beta = min(beta, best_score)

            # Alpha-Beta pruning: cut off search if beta ≤ alpha (only if not in easy mode)
            if not easy_mode and beta <= alpha:
                break
        return best_move, best_score

def AI_Player(board):
    win_move = oneWinMove(board, 2)
    if not easy_mode and depth == 4 and win_move:
        return win_move

    block_move = oneWinMove(board, 1)
    if not easy_mode and depth == 4 and block_move:
        return block_move

    move, _ = minimax(board, depth, 4, -100000, 100000, easy_mode=False)
    return move

def generateBoard(board):
    for i in range(gameRows):  # Loop through each row
        col_list = []  # Create a new row (list of columns)
        for j in range(gameCols):  # Loop through each column
            col_list.append(0)  # Initialize with 0 (empty cell)
        board.append(col_list)  # Add the row to the board

def redrawBoard(window):
    window.fill(backgroundColor)  # Clear the window with background color

    for i in range(gameRows):
        # Draw horizontal grid lines
        pygame.draw.line(window, blackLine, (cellCenter, i*cellSize + cellCenter), (width - cellCenter, i*cellSize + cellCenter), 1)
    for j in range(gameCols):
        # Draw vertical grid lines
        pygame.draw.line(window, blackLine, (j*cellSize + cellCenter, cellCenter), (j*cellSize + cellCenter, height - cellCenter), 1)

def redrawRocks(window):
    global board
    for i in range(gameRows):
        for j in range(gameCols):
            if board[i][j] == 1:
                rockColor = player_color  # Player piece color
            elif board[i][j] == 2:
                rockColor = AI_player  # AI piece color
            if board[i][j] != 0:
                # Draw circle at cell center
                pygame.draw.circle(window, rockColor, (j*cellSize + cellCenter, i*cellSize + cellCenter), cellCenter - 5)

def menuButton(window):
    x = gameCols * cellSize + 20
    y = 250
    endX = 160
    endY = 40
    pygame.draw.rect(window, (250, 250, 250), (x, y, endX, endY), 0, 10)  # Draw button
    text = font.render("Main menu", True, (0, 0, 0))  # Button text
    window.blit(text, (x + 20, y + 8))  # Draw text
    return x, y, endX, endY  # Return button bounds

def redrawSidePanel(window, winner):
    pygame.draw.rect(window, (220, 200, 200), (gameCols*cellSize, 0, 200, height//1.5), 0)  # Side panel background

    # Show game status message
    if winner == 1:
        msg = "player win"
    elif winner == 2:
        msg = "AI win"
    elif drawResult(board):
        msg = "No solution , Draw !"
    elif winner == 0:
        msg = f"Turn: player {'' if turn == 1 else 'AI'}"

    text = font.render(msg, True, (0, 0, 0))
    window.blit(text, (gameCols * cellSize + 20, 20))

    # Display board size
    size = f"Board: {gameRows} × {gameCols}"
    sizeText = font.render(size, True, (255, 100, 100))
    window.blit(sizeText, (gameCols * cellSize + 20, 50))

    # Draw restart button
    x = gameCols * cellSize + 20
    y = 100
    endX = 160
    endY = 40
    pygame.draw.rect(window, (250, 250, 250), (x, y, endX, endY), 0, 10)
    restText = font.render("restart", True, (0, 10, 250))
    window.blit(restText, (gameCols * cellSize + 70, 105))
    return x, y, endX, endY

def increaseButtons(window):
    x = gameCols * cellSize + 20
    y = 150
    endX = 50
    endY = 50
    pygame.draw.rect(window, (255, 255, 255), (x, y, endX, endY), 0, 5)
    increaseSign = font.render("+", True, (0, 10, 250))
    window.blit(increaseSign, (x + 18, y + 9))
    return x, y, endX, endY

def decreaseButtons(window):
    x = gameCols * cellSize + 100
    y = 150
    endX = 50
    endY = 50
    pygame.draw.rect(window, (255, 255, 255), (x, y, endX, endY), 0, 5)
    increaseSign = font.render("-", True, (0, 10, 250))
    window.blit(increaseSign, (x + 20, y + 9))
    return x, y, endX, endY

def resize(window, change):
    global gameCols, gameRows, width, height, windowWidth
    gameCols += change
    gameRows += change
    width = gameRows * cellSize
    height = gameCols * cellSize
    windowWidth = width + 200  # Side panel
    return pygame.display.set_mode((windowWidth, height))  # Update screen size

def getClickPostition(pos):
    x, y = pos
    col = x // cellSize
    row = y // cellSize
    return row, col

def checkAntiDiagonal(row, col, type):
    count = 1
    
    # Check down-right direction
    next_row, next_col = row + 1, col + 1
    while next_row < gameRows and next_col < gameCols and board[next_row][next_col] == type:
        count += 1
        next_row += 1
        next_col += 1

    # Check up-left direction
    prev_row, prev_col = row - 1, col - 1
    while prev_row >= 0 and prev_col >= 0 and board[prev_row][prev_col] == type:
        count += 1
        prev_row -= 1
        prev_col -= 1
    
    return 1 if count >= 5 else 0

def checkDiagonal(row, col, type):
    count = 1
    
    # Check down-left direction
    next_row, prev_col = row + 1, col - 1
    while next_row < gameRows and prev_col >= 0 and board[next_row][prev_col] == type:
        count += 1
        next_row += 1
        prev_col -= 1

    # Check up-right direction
    prev_row, next_col = row - 1, col + 1
    while prev_row >= 0 and next_col < gameCols and board[prev_row][next_col] == type:
        count += 1
        prev_row -= 1
        next_col += 1
    
    return 1 if count >= 5 else 0

def checkVertical(row, col, type):
    count = 1

    # Check downward
    next_row = row + 1
    while next_row < gameRows and board[next_row][col] == type:
        count += 1
        next_row += 1

    # Check upward
    prev_row = row - 1
    while prev_row >= 0 and board[prev_row][col] == type:
        count += 1
        prev_row -= 1
        
    return 1 if count >= 5 else 0

def checkHorizontal(row, col, type):
    count = 1
    
    # Check right
    next_col = col + 1
    while next_col < gameCols and board[row][next_col] == type:
        count += 1
        next_col += 1

    # Check left
    prev_col = col - 1
    while prev_col >= 0 and board[row][prev_col] == type:
        count += 1
        prev_col -= 1

    return 1 if count >= 5 else 0

def checkWinner(board):
    for row in range(gameRows):
        for col in range(gameCols):
            current = board[row][col]
            if current == 0:
                continue
            if (checkVertical(row, col, current) or
                checkHorizontal(row, col, current) or
                checkDiagonal(row, col, current) or
                checkAntiDiagonal(row, col, current)):
                return current  # Return winning player
    return 0  # No winner

pygame.init()
gameWindow = pygame.display.set_mode((windowWidth, height))
pygame.display.set_caption("Animus_Gomoku")
icon = pygame.image.load("gameLogo.png")
pygame.display.set_icon(icon)
numOfFrames = pygame.time.Clock()
board = []
generateBoard(board)
turn = 1

while True:  # Start the main game loop (runs continuously)
    if not isMenu:  # If we're not on the main menu (i.e., we are in the game)
        winner = checkWinner(board)  # Check if there's a winner
        winner = checkWinner(board)  # (Unnecessary duplicate – you can remove one)
        
        # Reset button boundaries in the side panel
        buttonX, buttonY, limitX, LimitY = redrawSidePanel(gameWindow, winner)
        
        # Increase board size button boundaries
        increaseX, increaseY, endX, endY = increaseButtons(gameWindow)
        
        # Decrease board size button boundaries
        decreaseX, decreaseY, endXD, endYD = decreaseButtons(gameWindow)
        
        # Main menu button boundaries
        mainX, mainY, endButtonX, endButtonY = menuButton(gameWindow)
        
        for event in pygame.event.get():  # Get all user input events
            
            # Player's turn (Human)
            if event.type == pygame.MOUSEBUTTONDOWN and winner == 0 and turn == 1:
                print(checkWinner(board))  # Debug print of winner
                currentRow, currentCol = getClickPostition(pygame.mouse.get_pos())  # Get clicked cell
                # If inside board and cell is empty
                if currentRow < gameRows and currentCol < gameCols and board[currentRow][currentCol] == 0:
                    board[currentRow][currentCol] = turn  # Place player's piece
                    turn = 2 if turn == 1 else 1  # Switch turn to AI
            
            # AI Turn
            elif turn == 2 and winner == 0:
                move = AI_Player(board)  # Get AI move
                if move:
                    currentRow, currentCol = move
                    board[currentRow][currentCol] = turn  # Place AI piece
                    turn = 1  # Switch back to player's turn
            
            # Handle Mouse Clicks for Buttons
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()  # Get mouse position
                
                # Restart button clicked
                if buttonX <= x < buttonX + limitX and buttonY <= y < buttonY + LimitY:
                    board.clear()  # Clear current board
                    generateBoard(board)  # Create fresh board
                    turn = 1  # Player starts
                
                # Increase board size (+ button clicked)
                elif increaseX <= x < increaseX + endX and increaseY <= y < increaseY + endY:
                    if 5 <= gameCols <= 19 and 5 <= gameRows < 19:
                        resize(gameWindow, 1)  # Increase board size
                        turn = 1
                        board.clear()
                        generateBoard(board)
                
                # Decrease board size (- button clicked)
                elif decreaseX <= x < decreaseX + endXD and decreaseY <= y < decreaseY + endYD:
                    if 5 < gameCols < 19 and 5 < gameRows < 19:
                        resize(gameWindow, -1)  # Decrease board size
                        turn = 1
                        board.clear()
                        generateBoard(board)
                
                # Main menu button clicked
                elif mainX <= x < mainX + endButtonX and mainY <= y < mainY + endButtonY:
                    resize(gameWindow, 15 - gameCols)  # Reset board to 15×15
                    board.clear()
                    generateBoard(board)
                    turn = 1
                    isMenu = 1  # Switch to main menu
            
            # Quit the game if user closes the window
            elif event.type == pygame.QUIT:
                pygame.quit()  # Quit Pygame
                sys.exit()  # Exit program
        
        # Redraw all game elements
        redrawBoard(gameWindow)  # Draw grid
        redrawRocks(gameWindow)  # Draw pieces
        redrawSidePanel(gameWindow, winner)  # Update side panel
        increaseButtons(gameWindow)  # Re-draw + button
        decreaseButtons(gameWindow)  # Re-draw – button
        menuButton(gameWindow)  # Re-draw menu button
    
    else:  # If we're in the main menu
        xButton, easyButton, medieumButton, hardButton = startMenu(gameWindow)  # Show menu
        
        for event in pygame.event.get():  # Handle menu interactions
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                
                # Easy mode selected
                if xButton < x < xButton + 120 and easyButton < y < easyButton + 50:
                    easy_mode = True
                    depth = 2  # Shallow AI
                    isMenu = False  # Start game
                
                # Medium mode selected
                elif xButton < x < xButton + 120 and medieumButton < y < medieumButton + 50:
                    depth = 3  # Medium AI
                    isMenu = False
                    easy_mode = False
                
                # Hard mode selected
                elif xButton < x < xButton + 120 and hardButton < y < hardButton + 50:
                    easy_mode = False
                    depth = 4  # Harder AI
                    isMenu = False
            
            # Quit the game if user closes the window
            elif event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
    
    # Final updates (refresh the screen every frame)
    pygame.display.flip()  # Update the display with new drawings
    numOfFrames.tick(60)  # Limit game to 60 frames per second